# CLEAN UP CODE for JAN 8 MEETING 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math 
products_static = pd.read_csv("products_static_1607946056.csv")

# Inputs: 
Input the values you would like to filter by:

In [2]:
#Filter 1: Categories
(cat_1_on, cat_2_on, cat_3_on) = False, True, False
cat_lst_1 = []
cat_lst_2 = [12999, 1301, 21762, 1297, 1295, 17095, 893, 
              17089, 12738, 2037,2035, 9197, 20768, 2033, 
              1329, 14492, 14503, 21849, 16502, 16500]
cat_lst_3 = []

#Filter 2: Times (Update-status, how established product is)
time_on = True
unix_time_now = 1607946056 #Change based on Nick's input
min_years = 1
updated_previously_weeks = 2


#Filter 3: Brand GMV
min_gmv_on = True
min_gmv = 10000

#Filter 4: Brand Avg Price 
min_unit_price_on = True
min_unit_price = 300

#Filter 5: GMV Concentration
min_gmv_conc_on = False
top_n = 3 #concetration num of products #IF CHANGE THIS NUMBER CHANGE COLUMN NAME TOO 
min_concentration = 0.2

#Filter 6: Avg Product Rating
min_avg_star_on = False
min_avg_star = 4.5

#Filter 7: Bad Customer Ratings 
max_bad_rating_on = False 
max_bad_rating = 0.5

#Filter 8: Brand TOTAL GMV 
min_total_brand_gmv_on = True
min_total_brand_gmv = 2000000 / 12

# Import Data

In [3]:
shops_gmv = pd.read_csv("shops_gmv_data.csv")
products_static = pd.read_csv("products_static_1607946056.csv")
models_static = pd.read_csv("models_static_data.csv")

Change data types in dataframes to conserve memory:

In [4]:
def convert_to_cats(df, to_convert):
    '''
    Converts cols in to_convert_list to categories. 
    
    Inputs:
        df: a DataFrame
        to_convert: a list of column names (as strings)
                    we'd like to convert
    Returns:
        Nothing; modifies dataframe in place
    '''
    for col in df.columns:
        if col in to_convert:
            df[col] = df[col].astype("category")

In [5]:
to_convert_models = ['category_one', 'category_one_en',
                     'category_one_th', 'category_two',
                     'category_two_en', 'category_two_th',
                     'category_three', 'category_three_en',
                     'category_three_th']
convert_to_cats(models_static, to_convert_models)

to_convert_products = ['category_one', 'category_one_en',
                      'category_one_th', 'category_two', 
                      'category_two_en', 'category_two_th',
                      'category_three', 'category_three_en', 
                      'category_three_th', 'reviews_count_context', 
                      'reviews_count_image', 'shopee_verified', 
                      'show_discount']
convert_to_cats(products_static, to_convert_products)

to_convert_shops = ["shopid", "category_one", "category_two",
                    "category_three"]
convert_to_cats(shops_gmv, to_convert_shops)

In [6]:
def downcast_numbers(df):
    '''
    Downcasts floats and ints.
    
    Inputs:
        df: a DataFrame objecet. 
    Returns:
        Nothing; modifies list in place
    '''
    for col in df.columns:
        if df[col].dtype == "float":
            df[col] = pd.to_numeric(df[col], downcast="float")
        if df[col].dtype == "int":
            df[col] = pd.to_numeric(df[col], downcast="unsigned")

In [7]:
downcast_numbers(models_static)
downcast_numbers(products_static)
downcast_numbers(shops_gmv)

#Making untouched, but memory efficient copy of data for later
products_static["product_gmv"] = products_static["price"] * products_static["sold"] 
products_static_untouched = products_static.copy(deep=True)


# Filter 1: Category Filter

Filter our DataFrames to only keep rows with Categories we care for. 


In [8]:
#Helper Functions
def filter_by_category(df, category_n, ok_cat_lst):
    '''
    Keeps rows in the df that have cats in ok_cat_lst. 
    
    Inputs:
        df: a DataFrame
        category_n: (str) eg. "category_one"
        ok_cat_lst: (lst) of acceptable cat for that 
          cat as numbers eg [50, 26]
    Returns:
        Nothing; modifies list in place
    '''
    
    mask = df[category_n].isin(ok_cat_lst)
    df = df[mask]
    
def filter_by_category(df, category_n, ok_cat_lst):
    '''
    Keeps rows in the df that have cats in ok_cat_lst. 
    
    Inputs:
        df: a DataFrame
        category_n: (str) eg. "category_one"
        ok_cat_lst: (lst) of acceptable cat for that 
          cat as numbers eg [50, 26]
    Returns:
        The modified df 
    '''
    
    mask = df[category_n].isin(ok_cat_lst)
    return df[mask]

In [9]:
if cat_1_on:
    models_static = filter_by_category(models_static, "category_one", cat_lst_1)
    products_static = filter_by_category(products_static, "category_one", cat_lst_1) 

In [10]:
if cat_2_on:
    models_static = filter_by_category(models_static, "category_two", cat_lst_2)
    products_static = filter_by_category(products_static, "category_two", cat_lst_2) 

In [11]:
if cat_3_on:
    models_static = filter_by_category(models_static, "category_three", cat_lst_3)
    products_static = filter_by_category(products_static, "category_three", cat_lst_3) 

# Filter 2: Keep only Established Products and Products that Sellers are Updating

Filter out products that have been sold for less than a specified time. 

In [12]:
def filter_out_new_products(df, min_years, unix_time_now):
    '''
    Filters out rows from our products_static df that 
    have been sold less than a specified time
    '''

    min_s = min_years * 365 * 24 * 60 * 60 

    mask = (unix_time_now - products_static["ctime"] > min_s)

    return df[mask]

def filter_out_unupdated_products(df, updated_previously_weeks):
    '''
    Filters out rows from our products_static df that 
    have been sold less than a specified time
    '''

    max_s = updated_previously_weeks * 7 * 24 * 60 * 60

    mask = (unix_time_now - products_static["modified_at"] < max_s)

    return df[mask]

if time_on: 
    products_static = filter_out_new_products(products_static, min_years,
                                              unix_time_now)

    products_static = filter_out_unupdated_products(products_static, 
                                                    updated_previously_weeks)

# Brand Definition

Extract unique brand names from the remaining products

In [13]:
mask = products_static["brand"] != "No Brand(ไม่มียี่ห้อ)"
products_static = products_static[mask]

brands_list = products_static["brand"].unique()
brands_list = np.delete(brands_list, np.where(brands_list == ('No Brand(ไม่มียี่ห้อ)')))

In [14]:
#Clean up brands_df
brands_df = pd.DataFrame(brands_list, columns = ["Brand Name"])

brands_df.dropna(inplace=True)

s1 = brands_df["Brand Name"] != "None"
brands_df = brands_df[s1]

s2 = brands_df["Brand Name"] != "0"
brands_df = brands_df[s2] 


In [15]:
#Set up columns for our brands_df

#NOTE: these values are all 1 month-based since the products data was only one month
brands_df["Filtered_Brand_GMV"] = 0 
brands_df["Filtered_Average_Unit_Price"] = 0
brands_df["%_Filtered_GMV_from_Top_3_Products"] = 0
brands_df["Filtered_Weighted_Star_Rating"] = 0
brands_df["Filtered_Bad_Rating_Percent"] = 0
brands_df.set_index("Brand Name", inplace = True)

In [16]:
#Make new column in products_static for GMV of a product (price * sold)
products_static["weighted_star"] = products_static["rating_star"] * products_static["sold"]

In [17]:
for brand in brands_df.index:
    
    #make a sub-df containing only rows with the correct brand
    my_brand_df = products_static[products_static["brand"] == brand]
    my_brand_df = my_brand_df.sort_values(by=["product_gmv"], ascending=False) 
    
    #gmv calculation
    brand_gmv = my_brand_df["product_gmv"].sum()
    brands_df.loc[brand, "Filtered_Brand_GMV"] = brand_gmv
    
    #volume calculation
    brand_volume = my_brand_df["sold"].sum()
    brands_df.loc[brand, "Filtered_Average_Unit_Price"] = brand_gmv / brand_volume
    
    #GMV Concentration Calculation
    if len(my_brand_df) >= top_n: 
        top_n_df = my_brand_df.head(top_n)
        top_n_gmv = top_n_df["product_gmv"].sum()
        brands_df.loc[brand, "%_Filtered_GMV_from_Top_3_Products"] = top_n_gmv / brand_gmv
    else:
        #brand sells less than like 5 products --> highly conc.
        brands_df.loc[brand, "%_Filtered_GMV_from_Top_3_Products"] = 1.0 
        
        
    #Weighted Star Rating
    total_star = my_brand_df["weighted_star"].sum()
    brands_df.loc[brand, "Filtered_Weighted_Star_Rating"] = total_star / brand_volume
    
    #Bad Rating Count
    bad_rating_count = my_brand_df["rating_count_one"].sum() + my_brand_df["rating_count_two"].sum()
    total_rating_count = my_brand_df["rating_count_total"].sum()
    
    if total_rating_count != 0:
        val = bad_rating_count / total_rating_count 
    else: 
        val = -1
    
    brands_df.loc[brand, "Filtered_Bad_Rating_Percent"] = val

brands_df["Filtered_Bad_Rating_Percent"] = brands_df["Filtered_Bad_Rating_Percent"].round(decimals=3)

# Filter 3: Brand Total GMV

In [18]:
if min_gmv_on:
    brands_df = brands_df[brands_df["Filtered_Brand_GMV"] >= min_gmv]

# Filter 4: Brand Average Price

In [19]:
if min_unit_price_on:
    brands_df = brands_df[brands_df["Filtered_Average_Unit_Price"] >= min_unit_price]

# Filter 5: GMV Concentration of Top (3) Products

In [20]:
if min_gmv_conc_on:
    brands_df = brands_df[brands_df["Filtered_GMV_Concentration"] >= min_concentration]

# Filter 6: Average Product Rating

In [21]:
if min_avg_star_on:
    brands_df = brands_df[brands_df["Filtered_Weighted_Star_Rating"] >= min_avg_star]

# Filter 7: Bad Customer Ratings

In [22]:
if max_bad_rating_on:
    brands_df = brands_df[brands_df["Filtered_Bad_Rating_Percent"] <= max_bad_rating]

# (GMV Growth?)

In [23]:
# number of brands in our brands_df
print(len(brands_df))

1145


# Filter 8: Brand GMV (Across All Categories) 

Use a fresh copy of the products data to get more info on our final list of brands. We use a fresh set because now that we've narrowed down our brand list, we want to see how our Brands are doing as a whole (all cats, all products). 

In [24]:
#Assume this is always kept on for simplicity
if min_total_brand_gmv_on:
        
    for brand in brands_df.index:

        #make a sub-df containing only rows with the correct brand
        mask_brand = products_static_untouched["brand"] == brand
        my_brand_df = products_static_untouched[mask_brand]

        #TOTAL Brand gmv calculation
        total_brand_gmv = my_brand_df["product_gmv"].sum()
        brands_df.loc[brand, "Brand_GMV_All_Cats"] = total_brand_gmv
        
        #Total brand view count 
        total_brand_views = my_brand_df["view_count"].sum()
        conv_rate = total_brand_gmv / total_brand_views
        brands_df.loc[brand, "Conversion_Rate_All_Cats_(total_gmv/view)"] = conv_rate
        
        #--------------------------------------------------------------
        #Take this out and put later if the min_total_brand_gmv 
        #becomes higher, otherwise inclue this in the for loop
        winning_cat_2 = None
        winning_cat_2_gmv = 0.0

        for cat_2 in my_brand_df["category_two_en"].unique():
            mask_cat_2 = my_brand_df["category_two_en"] == cat_2
            cat_2_df = my_brand_df[mask_cat_2]

            cat_2_gmv = cat_2_df["product_gmv"].sum()
            if cat_2_gmv > winning_cat_2_gmv:
                winning_cat_2 = cat_2
                winning_cat_2_gmv = cat_2_gmv
                    
        percent = (winning_cat_2_gmv / total_brand_gmv).round(decimals=3)
        brands_df.loc[brand, "Cat_2_Percent_of_Total_GMV"] = percent 
        brands_df.loc[brand, "Brand_Category_2"] = winning_cat_2

        #print("for brand: ", brand, "Winning cat 2 is: ", winning_cat_2)
        #print("with a cat_2_gmv value of: ", winning_cat_2_gmv)
        
        #--------------------------------------------------------------
            
        
        
    #Apply the total brand gmv filter
    brands_df = brands_df[brands_df["Brand_GMV_All_Cats"] >= min_total_brand_gmv]

# Calculate Additional Info on Recency

From the final list of brands we have, add info on recency ratio eg. percentage of total gmv coming from products established at least 3, 6, 12, 24 months. 

In [26]:
three_mo = 3 * 30 * 24 * 60 * 60
six_mo = 6 * 30 * 24 * 60 * 60
twelve_mo = 12 * 30 * 24 * 60 * 60
twenty_four_mo = 24 * 30 * 24 * 60 * 60

In [27]:
#Make four dataframes representing slices of the untouched products
#that have sold for more than 3, 6, 12, 24 months. 

mask_3 = unix_time_now - products_static_untouched["ctime"] > three_mo
timed_df_3 = products_static_untouched[mask_3]

mask_6 = unix_time_now - products_static_untouched["ctime"] > six_mo
timed_df_6 = products_static_untouched[mask_6]

mask_12 = unix_time_now - products_static_untouched["ctime"] > twelve_mo
timed_df_12 = products_static_untouched[mask_12]

mask_24 = unix_time_now - products_static_untouched["ctime"] > twenty_four_mo
timed_df_24 = products_static_untouched[mask_24]

In [28]:
for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_3[timed_df_3["brand"] == brand]
    
    #gmv calculation
    brand_gmv_3 = my_brand_df["product_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 3 months+"] = brand_gmv_3
    

In [29]:
for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_6[timed_df_6["brand"] == brand]
    
    #gmv calculation
    brand_gmv_6 = my_brand_df["product_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 6 months+"] = brand_gmv_6

In [30]:
for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_12[timed_df_12["brand"] == brand]
    
    #gmv calculation
    brand_gmv_12 = my_brand_df["product_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 12 months+"] = brand_gmv_12

In [31]:
for brand in brands_df.index:
    #make a sub-df containing only rows with the correct brand
    my_brand_df = timed_df_24[timed_df_24["brand"] == brand]
    
    #gmv calculation
    brand_gmv_24 = my_brand_df["product_gmv"].sum()
    brands_df.loc[brand, "%Total GMV from products 24 months+"] = brand_gmv_24

In [32]:
#Now we have baht gmv in the four recency columns, 
#just convert into ratio compared to total_gmv 

In [33]:
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 3 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 3 months+"] = round(percentage, 3)

In [34]:
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 6 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 6 months+"] = round(percentage, 3)

In [35]:
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 12 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 12 months+"] = round(percentage, 3)

In [36]:
for brand in brands_df.index:
    percentage = brands_df.loc[brand, "%Total GMV from products 24 months+"] / brands_df.loc[brand, 
                                                                                   "Brand_GMV_All_Cats"]
    brands_df.loc[brand, "%Total GMV from products 24 months+"] = round(percentage, 3)

# Output: Brands that Meet Our Criterion

In [37]:
brands_df.sort_index(inplace=True)
display(brands_df)
#NOTE: Anything before the Brand_GMV_All_Cats uses FILTERED product data 
#i.e. data where we've the products to count for by category, and time. 

,Filtered_Brand_GMV,Filtered_Average_Unit_Price,%_Filtered_GMV_from_Top_3_Products,Filtered_Weighted_Star_Rating,Filtered_Bad_Rating_Percent,Brand_GMV_All_Cats,Conversion_Rate_All_Cats_(total_gmv/view),Cat_2_Percent_of_Total_GMV,Brand_Category_2,%Total GMV from products 3 months+,%Total GMV from products 6 months+,%Total GMV from products 12 months+,%Total GMV from products 24 months+
Brand Name,,,,,,,,,,,,,
3M,285462,313.350165,0.284048,4.778005,0.012,563616.0,6.032430,0.316,Car care products,0.999,0.977,0.932,0.724
ACCU-CHEK(แอคคิวเช็ค),684677,680.593439,0.665321,4.919250,0.002,1017173.0,38.517608,1.000,Health equipment,0.925,0.854,0.677,0.355
ACDelco(เอซีเดลโก้),18178,826.272727,0.775993,4.864057,0.002,333032.0,13.708970,0.909,Oil and liquid,0.964,0.797,0.468,0.358
AIKO(ไอโกะ),41810,459.450549,0.522865,4.782051,0.011,211626.0,6.813238,0.540,Gas stove,0.991,0.889,0.272,0.026
AJ(เอเจ),65799,715.206522,0.948571,4.570254,0.038,1573309.0,8.474780,0.302,Gas stove,0.829,0.751,0.645,0.257
...,...,...,...,...,...,...,...,...,...,...,...,...,...
sonar(โซนาร์),22349,698.406250,0.644548,4.663946,0.015,472764.0,12.167392,0.891,Refrigerator,1.000,0.996,0.970,0.020
xiaomi,52892,362.273973,1.000000,4.763268,0.017,697365.0,6.271381,0.252,Computer accessories,0.997,0.643,0.183,0.006
ดอกกรรณิการ์,359950,394.249726,0.886290,4.857046,0.006,359950.0,58.585612,1.000,Health equipment,1.000,1.000,1.000,0.000


In [38]:
#make to csv here if desired 
#brands_df.to_csv("test_3.csv")

# MISCELLANEOUS: Additional Work for Jan 8 Meeting

In [39]:
#Begin by importing data
products_static_raw = pd.read_csv("products_static_1607946056.csv")
products_static_raw["product_gmv"] = products_static_raw["price"] * products_static_raw["sold"] 

In [40]:
#Take Sheet Nakrin wants and add my column
info_1 = pd.read_csv("info_1_for_Jan_8.csv")

In [41]:
#Step 1: Convert Nakrin's given Cat 1's into Numbers as given by mapping
#Don't have to include thai. If really want to can always look up based
#on cat number
### 
category_ones = products_static_raw.loc[:, ("category_one_en", "category_one")]
category_ones.drop_duplicates(inplace=True)
cat_1_names = category_ones.sort_values(by="category_one_en")
### 
category_twos = products_static_raw.loc[:, ("category_two_en", "category_two")]
category_twos.drop_duplicates(inplace=True)
cat_2_names = category_twos.sort_values(by="category_two_en")
### 
category_threes = products_static_raw.loc[:, ("category_three_en", "category_three")]
category_threes.drop_duplicates(inplace=True)
cat_3_names = category_threes.sort_values(by="category_three_en")
'''
Call this if want a csv containing the most recent
"proper" cat names i.e. computer readable
cat_1_names.to_csv("cat_1_names.csv")
cat_2_names.to_csv("cat_2_names.csv")
cat_3_names.to_csv("cat_3_names.csv")
'''

#Manually delete rows we don't want from the cat list
#This will likely be a rough list of cats we want since
#we've eliminated any non attractive cats
cat_1_names.drop(cat_1_names.loc[cat_1_names["category_one"]==2085].index, 
                 inplace=True)
cat_1_names.dropna(inplace=True)

In [42]:
for cat_1 in info_1["Cat_1_computer"]:
    cat_1_mask = products_static_raw["category_one_en"] == cat_1
    cat_1_df = products_static_raw[cat_1_mask]
    cat_gmv = cat_1_df["product_gmv"].sum()
    info_1.loc[cat_1, "Annual GMV (Sales)"] = cat_gmv * 12
    

KeyError: 'Cat_1_computer'

In [ ]:
#info_1.to_csv("info_1_additional_info.csv")

### Repeat Process for Cat 2

In [ ]:
#Challenge is it's not only the cat 2 you care about 
#but which cat 1 that cat 2 is from. 

#i.e. need a way to pull both cat_1_computer_name eg. "Accessories"
# AND cat_2_computer_name eg. "Bracelet"

In [ ]:
info_2 = pd.read_csv("info_2_for_Jan_8.csv")

In [ ]:
info_2

In [ ]:
#can do a manual solution i.e. specify cat 1 then apply a 
#second mask for cat 2 filter eg. 
store_1 = ['Accessories', 'Bag', "Men's fashion", "Men's Shoes",
         "Women's fashion", "Women's Shoes"]
store_2 = ['Bracelet',
         'Hair accessory',
         'Hat',
         'Scarves and shawls',
         'Backpack',
         'Bag accessories',
         'Brand name',
         'Cloth bag',
         'Earrings',
         'Shoulder bag',
         'Backpack',
         'Bracelet',
         'Health equipment',
         'Jeans',
         "Men's underwear",
         'Other',
         'Outerwear',
         'Shorts',
         'T-shirt',
         'Big head shoes',
         'Boots',
         'Computer accessories',
         'Gaming equipment',
         'Heel strap shoes',
         'Lace up shoes',
         'Lace-up sneakers',
         'Slippers',
         'Snacks',
         'Socks',
         'Backpack',
         'Belt',
         'Denim',
         'Dress',
         'Health equipment',
         'Jackets and Coats',
         'Jumpsuit',
         "Men's underwear",
         'Pants',
         'Personal belongings',
         'Plus size clothing',
         'Shirt',
         'Skirt',
         'Swimwear',
         'T-shirt',
         'Underwear',
         "Women's sportswear",
         'Backpack',
         'Big head shoes',
         'Boots',
         'Canvas shoes',
         'Casual shoes',
         'Flat shoes',
         'Gaming equipment',
         'Health equipment',
         'Heel strap shoes',
         'High heels',
         'Shoulder bag']

In [ ]:
#START HERE: can do this manual process if want but 
#there should be a more elegant solution

In [ ]:
#more elegant attempt

cat_1 = "Accessories"
mask1 = products_static_raw["category_one_en"] == cat_1

for cat_2 in ['Bracelet',
         'Hair accessory',
         'Hat',
         'Scarves and shawls']:
    
    mask2 = products_static_raw["category_two_en"] == cat_2
    
    cat_2_df = products_static_raw[mask1 & mask2]
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    annual_gmv = cat_2_gmv * 12
    info_2.loc[cat_2, "Annual GMV (Sales)"] = annual_gmv

In [ ]:
#more elegant attempt

cat_1 = "Bag"
mask1 = products_static_raw["category_one_en"] == cat_1

for cat_2 in ['Backpack',
         'Bag accessories',
         'Brand name',
         'Cloth bag',
         'Earrings',
         'Shoulder bag']:
    
    mask2 = products_static_raw["category_two_en"] == cat_2
    
    cat_2_df = products_static_raw[mask1 & mask2]
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    annual_gmv = cat_2_gmv * 12
    info_2.loc[cat_2, "Annual GMV (Sales)"] = annual_gmv

In [ ]:
#more elegant attempt

cat_1 = "Men's fashion"
mask1 = products_static_raw["category_one_en"] == cat_1

for cat_2 in ['Backpack',
         'Bracelet',
         'Health equipment',
         'Jeans',
         "Men's underwear",
         'Other',
         'Outerwear',
         'Shorts',
         'T-shirt']:
    
    mask2 = products_static_raw["category_two_en"] == cat_2
    
    cat_2_df = products_static_raw[mask1 & mask2]
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    annual_gmv = cat_2_gmv * 12
    info_2.loc[cat_2, "Annual GMV (Sales)"] = annual_gmv

In [ ]:
#more elegant attempt

cat_1 = "Men's Shoes"
mask1 = products_static_raw["category_one_en"] == cat_1

for cat_2 in ['Big head shoes',
         'Boots',
         'Computer accessories',
         'Gaming equipment',
         'Heel strap shoes',
         'Lace up shoes',
         'Lace-up sneakers',
         'Slippers',
         'Snacks',
         'Socks']:
    
    mask2 = products_static_raw["category_two_en"] == cat_2
    
    cat_2_df = products_static_raw[mask1 & mask2]
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    annual_gmv = cat_2_gmv * 12
    info_2.loc[cat_2, "Annual GMV (Sales)"] = annual_gmv

In [ ]:
#more elegant attempt

cat_1 = "Women's fashion"
mask1 = products_static_raw["category_one_en"] == cat_1

for cat_2 in ['Backpack',
         'Belt',
         'Denim',
         'Dress',
         'Health equipment',
         'Jackets and Coats',
         'Jumpsuit',
         "Men's underwear",
         'Pants',
         'Personal belongings',
         'Plus size clothing',
         'Shirt',
         'Skirt',
         'Swimwear',
         'T-shirt',
         'Underwear',
         "Women's sportswear"]:
    
    mask2 = products_static_raw["category_two_en"] == cat_2
    
    cat_2_df = products_static_raw[mask1 & mask2]
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    annual_gmv = cat_2_gmv * 12
    info_2.loc[cat_2, "Annual GMV (Sales)"] = annual_gmv

In [ ]:
#more elegant attempt

cat_1 = "Women's Shoes"
mask1 = products_static_raw["category_one_en"] == cat_1

for cat_2 in ['Backpack',
         'Big head shoes',
         'Boots',
         'Canvas shoes',
         'Casual shoes',
         'Flat shoes',
         'Gaming equipment',
         'Health equipment',
         'Heel strap shoes',
         'High heels',
         'Shoulder bag']:
    
    mask2 = products_static_raw["category_two_en"] == cat_2
    
    cat_2_df = products_static_raw[mask1 & mask2]
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    annual_gmv = cat_2_gmv * 12
    info_2.loc[cat_2, "Annual GMV (Sales)"] = annual_gmv

In [ ]:
info_2.tail(57)

In [ ]:
#info_2.to_csv("info_2_additional_info.csv")

In [ ]:
category_twos = products_static_raw.loc[:, ("category_two_en", "category_two")]
category_twos.drop_duplicates(inplace=True)
cat_2_names = category_twos.sort_values(by="category_two_en")

In [ ]:
cat_2_names

In [ ]:
#check the cat 2's we didn't get to after
#running through the whole thing

In [ ]:
#Step 4: add Data to Sheet 3, sheet 5 on excel online

#Step 5: Do the trailing products ratio based on recency etc. 

#Step 6: plus reply to email

#address concern of brand gmv filter not working (it does, 
#see details) 

#and give this additional info

In [ ]:
#Don't need to create a new csv for name mappings b/c the level 2 is all good already
'''
category_twos = products_static_raw.loc[:, ("category_one_en", "category_two_en", "category_two")]
category_twos.drop_duplicates(inplace=True)
cat_2_names = category_twos.sort_values(by="category_two_en")

cat_2_names.sort_values(by=["category_one_en", "category_two_en"], inplace=True)

cat_2_names.to_csv("cat_2_computer_names.csv")
'''

In [ ]:
'''
To get category name info
models_static = pd.read_csv("models_static_data.csv")

mapping_1 = models_static[["category_one", "category_one_en", "category_one_th"]].drop_duplicates()
mapping_1 = mapping_1.set_index("category_one").dropna().sort_index()

mapping_2 = models_static[["category_two", "category_two_en", "category_two_th"]].drop_duplicates()
mapping_2 = mapping_2.set_index("category_two").dropna().sort_index()

mapping_3 = models_static[["category_three", "category_three_en", "category_three_th"]].drop_duplicates()
mapping_3 = mapping_3.set_index("category_three").dropna().sort_index()

mapping_1.to_csv("mapping_1.csv")
mapping_2.to_csv("mapping_2.csv")
mapping_3.to_csv("mapping_3.csv")

*NOTE: This gives a limited list b/c the models_static csv 
       was limited to just fashion cats last time (to make
       data transfer via harddrive easier). 
'''